**Assignment 2 & 3**

**QUESTION 1**

Kisaa Fatima

TO understand the and converting the parquet file

In [2]:
import pyarrow.parquet as pq

# Specify the path to your Parquet file
parquet_file_path = 'UNSW_NB15_testing-set.parquet'

# Read the Parquet file into a PyArrow Table
table = pq.read_table(parquet_file_path)

# Convert the PyArrow Table to a Pandas DataFrame if needed
df = table.to_pandas()

# Now you can work with the DataFrame
print(df.head())


        dur proto service state  spkts  dpkts  sbytes  dbytes       rate  \
0  0.121478   tcp       -   FIN      6      4     258     172  74.087486   
1  0.649902   tcp       -   FIN     14     38     734   42014  78.473373   
2  1.623129   tcp       -   FIN      8     16     364   13186  14.170161   
3  1.681642   tcp     ftp   FIN     12     12     628     770  13.677108   
4  0.449454   tcp       -   FIN     10      6     534     268  33.373825   

          sload  ...  trans_depth  response_body_len  ct_src_dport_ltm  \
0  14158.942383  ...            0                  0                 1   
1   8395.112305  ...            0                  0                 1   
2   1572.271851  ...            0                  0                 1   
3   2740.178955  ...            0                  0                 1   
4   8561.499023  ...            0                  0                 2   

   ct_dst_sport_ltm  is_ftp_login  ct_ftp_cmd  ct_flw_http_mthd  \
0                 1            

In [ ]:
#for the column names
print("Column Names:")
print(df.columns)

Column Names:
Index(['dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes',
       'dbytes', 'rate', 'sload', 'dload', 'sloss', 'dloss', 'sinpkt',
       'dinpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt',
       'synack', 'ackdat', 'smean', 'dmean', 'trans_depth',
       'response_body_len', 'ct_src_dport_ltm', 'ct_dst_sport_ltm',
       'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'is_sm_ips_ports',
       'attack_cat', 'label'],
      dtype='object')


For self evaluation using training data for 1 experiment

**FITNESS FUNC**
evaluates the fitness of a chromosome by considering the selected features, converting categorical variables, and calculating a fitness score based on the sum of selected feature values in the dataset

In [3]:
import pandas as pd
import numpy as np

# Load the training dataset
df_train = pd.read_parquet('UNSW_NB15_training-set.parquet')


# Define chromosome representation
def create_chromosome(num_features):
    return np.random.choice([0, 1], size=num_features)

def fitness_function(chromosome, df):
    # Identify selected features
    selected_features = df.columns[chromosome == 1]

    # Filter the dataframe to include only the selected features
    selected_df = df[selected_features]

    # Convert categorical variables to numerical representations
    selected_df = pd.get_dummies(selected_df, columns=selected_df.select_dtypes(include='category').columns)

    # Calculate fitness based on the sum of selected features
    fitness = selected_df.sum().sum()

    return fitness


**SELECTION:**

-selects individuals (chromosomes) from the population for next generation based on their fitness values.
-individuals with higher fitness values have a higher chance of being selected.
-The probabilities variable contains the probability of selecting each individual, which is proportional to its fitness value.

**MUTATION:**
 random changes to the chromosome by flipping individual bits according to the specified mutation rate


In [4]:
# Genetic operators
def selection(population, fitness_values):
    probabilities = fitness_values / np.sum(fitness_values)
    # np.random.choice function is then used to select individuals randomly according to these probabilities.
    selected_indices = np.random.choice(len(population), size=len(population), p=probabilities)
    selected_population = [population[i] for i in selected_indices]
    return selected_population
#crossover operator generates new individuals by combining genetic material from selected parents
def crossover(parent1, parent2):
    crossover_point = np.random.randint(1, len(parent1))
    child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
    return child1, child2

def mutation(chromosome, mutation_rate):
    for i in range(len(chromosome)):
        if np.random.random() < mutation_rate:
            chromosome[i] = 1 - chromosome[i]  # Flip the bit
    return chromosome


**POPULATION:** Each chromosome represents potential solution where each gene in the chromosome corresponds to whether a feature is selected or not.

**algo**: It have population initialization, evaluation, selection, crossover, mutation, and termination conditions, to search for an optimal solution for feature selection in a datas

In [5]:
# Initialize population
def initialize_population(population_size, num_features):
    population = [create_chromosome(num_features) for _ in range(population_size)]
    return population

# Genetic algorithm
def genetic_algorithm(df, num_generations, population_size, mutation_rate):
    num_features = len(df.columns)
    population = initialize_population(population_size, num_features)

    for generation in range(num_generations):
        fitness_values = [fitness_function(chromosome, df) for chromosome in population]
        selected_population = selection(population, fitness_values)
        offspring = []

        for i in range(0, len(selected_population), 2):
            parent1, parent2 = selected_population[i], selected_population[i+1]
            child1, child2 = crossover(parent1, parent2)
            offspring.extend([child1, child2])

        mutated_offspring = [mutation(chromosome, mutation_rate) for chromosome in offspring]
        population = mutated_offspring

    final_fitness_values = [fitness_function(chromosome, df) for chromosome in population]
    best_index = np.argmax(final_fitness_values)
    best_solution = population[best_index]
    best_fitness = final_fitness_values[best_index]

    return best_solution, best_fitness


In [ ]:

#example parameters
population_size = 100
num_generations = 50
mutation_rate = 0.01

#run the genetic algorithm
best_solution, best_fitness = genetic_algorithm(df_train, num_generations, population_size, mutation_rate)

print("Best solution:", best_solution)
print("Best fitness:", best_fitness)


Best solution: [0 1 1 0 0 1 1 1 1 1 1 0 0 0 1 1 0 1 1 1 1 0 0 1 1 0 0 0 1 0 1 0 1 0 1 1]
Best fitness: 183056667557483.72


with 3 different experiment

In [ ]:
import pandas as pd
import numpy as np

# Define chromosome representation
def create_chromosome(num_features):
    return np.random.choice([0, 1], size=num_features)

# Define fitness function
def fitness_function(chromosome, df):
    selected_features = df.columns[chromosome == 1]
    fitness = df[selected_features].sum().sum()
    return fitness

# Define selection function
def selection(population, fitness_values):
    probabilities = fitness_values / np.sum(fitness_values)
    selected_indices = np.random.choice(len(population), size=len(population), p=probabilities)
    selected_population = [population[i] for i in selected_indices]
    return selected_population

# Define crossover function
def one_point_crossover(parent1, parent2):
    crossover_point = np.random.randint(1, len(parent1))
    child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
    return child1, child2

def two_point_crossover(parent1, parent2):
    crossover_points = sorted(np.random.choice(len(parent1), size=2, replace=False))
    child1 = np.concatenate((parent1[:crossover_points[0]], parent2[crossover_points[0]:crossover_points[1]], parent1[crossover_points[1]:]))
    child2 = np.concatenate((parent2[:crossover_points[0]], parent1[crossover_points[0]:crossover_points[1]], parent2[crossover_points[1]:]))
    return child1, child2

# Define mutation function
def mutation(chromosome, mutation_rate):
    for i in range(len(chromosome)):
        if np.random.random() < mutation_rate:
            chromosome[i] = 1 - chromosome[i]
    return chromosome

# Initialize population
def initialize_population(population_size, num_features):
    population = [create_chromosome(num_features) for _ in range(population_size)]
    return population

# Genetic algorithm
def genetic_algorithm(df, num_generations, population_size, mutation_rate, crossover_type):
    num_features = len(df.columns)
    population = initialize_population(population_size, num_features)

    crossover_fn = one_point_crossover if crossover_type == 'one-point' else two_point_crossover

    for generation in range(num_generations):
        fitness_values = [fitness_function(chromosome, df) for chromosome in population]
        selected_population = selection(population, fitness_values)

        offspring = []
        for i in range(0, len(selected_population), 2):
            parent1, parent2 = selected_population[i], selected_population[i+1]
            child1, child2 = crossover_fn(parent1, parent2)
            offspring.extend([child1, child2])

        mutated_offspring = [mutation(chromosome, mutation_rate) for chromosome in offspring]

        population = mutated_offspring

    final_fitness_values = [fitness_function(chromosome, df) for chromosome in population]
    best_index = np.argmax(final_fitness_values)
    best_solution = population[best_index]
    best_fitness = final_fitness_values[best_index]

    return best_solution, best_fitness

# Preprocessing function to handle categorical columns
def preprocess_data(df):
    categorical_columns = df.select_dtypes(include=['category']).columns
    df = df.drop(categorical_columns, axis=1)
    return df

if __name__ == "__main__":
    # Load training dataset
    df_train = pd.read_parquet('UNSW_NB15_training-set.parquet')

    # Preprocess the training dataset
    df_train = preprocess_data(df_train)

    # Define parameters for experiments
    population_sizes = [50, 100, 200]
    crossover_types = ['one-point', 'two-point']
    mutation_rates = [0.01, 0.05, 0.1]
    num_generations = 50

    # Run experiments
    results = []
    for pop_size in population_sizes:
        for crossover_type in crossover_types:
            for mutation_rate in mutation_rates:
                best_solution, best_fitness = genetic_algorithm(df_train, num_generations, pop_size, mutation_rate, crossover_type)
                results.append({'Population Size': pop_size, 'Crossover Type': crossover_type, 'Mutation Rate': mutation_rate,
                                'Best Solution': best_solution, 'Best Fitness': best_fitness})

    # Save results to a CSV file
    results_df = pd.DataFrame(results)
    results_df.to_csv('exp_results.csv', index=False)


below is the code with 3 experiments, but using fitness function of the paper .

In [ ]:
import pandas as pd
import numpy as np

# chromosome representation
def create_chromosome(num_features):
    return np.random.choice([0, 1], size=num_features)

def fitness_function(chromosome, df, weights):
    #Calculate fitness based on the size & weight of selected features
    selected_features = df.columns[chromosome == 1]
    selected_indices = [df.columns.get_loc(feature) for feature in selected_features]
    size = df.iloc[:, selected_indices].sum().sum()  #sum of sizes of selected features
    #multiply by the sum of weights of selected features
    fitness = size * weights[selected_indices].sum()

    return fitness

#selection function
def selection(population, fitness_values):
    probabilities = fitness_values / np.sum(fitness_values)
    selected_indices = np.random.choice(len(population), size=len(population), p=probabilities)
    selected_population = [population[i] for i in selected_indices]
    return selected_population

#crossover function
def one_point_crossover(parent1, parent2):
    crossover_point = np.random.randint(1, len(parent1))
    child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
    return child1, child2

def two_point_crossover(parent1, parent2):
    crossover_points = sorted(np.random.choice(len(parent1), size=2, replace=False))
    child1 = np.concatenate((parent1[:crossover_points[0]], parent2[crossover_points[0]:crossover_points[1]], parent1[crossover_points[1]:]))
    child2 = np.concatenate((parent2[:crossover_points[0]], parent1[crossover_points[0]:crossover_points[1]], parent2[crossover_points[1]:]))
    return child1, child2

#for mutation function
def mutation(chromosome, mutation_rate):
    for i in range(len(chromosome)):
        if np.random.random() < mutation_rate:
            chromosome[i] = 1 - chromosome[i]
    return chromosome

#initializing population
def initialize_population(population_size, num_features):
    population = [create_chromosome(num_features) for _ in range(population_size)]
    return population

#genetic algorithm
def genetic_algorithm(df, num_generations, population_size, mutation_rate, crossover_type, weights):
    num_features = len(df.columns)
    population = initialize_population(population_size, num_features)

    crossover_fn = one_point_crossover if crossover_type == 'one-point' else two_point_crossover

    for generation in range(num_generations):
        fitness_values = [fitness_function(chromosome, df, weights) for chromosome in population]
        selected_population = selection(population, fitness_values)

        offspring = []
        for i in range(0, len(selected_population), 2):
            parent1, parent2 = selected_population[i], selected_population[i+1]
            child1, child2 = crossover_fn(parent1, parent2)
            offspring.extend([child1, child2])

        mutated_offspring = [mutation(chromosome, mutation_rate) for chromosome in offspring]

        population = mutated_offspring

    final_fitness_values = [fitness_function(chromosome, df, weights) for chromosome in population]
    best_index = np.argmax(final_fitness_values)
    best_solution = population[best_index]
    best_fitness = final_fitness_values[best_index]

    return best_solution, best_fitness

#preprocessing function to handle categorical columns
def preprocess_data(df):
    categorical_columns = df.select_dtypes(include=['category']).columns
    df = df.drop(categorical_columns, axis=1)
    return df

if __name__ == "__main__":
    #load training dataset
    df_train = pd.read_parquet('UNSW_NB15_training-set.parquet')

    #preprocess the training dataset
    df_train = preprocess_data(df_train)

    #define parameters for experiments
    population_sizes = [50, 100, 200]
    crossover_types = ['one-point', 'two-point']
    mutation_rates = [0.01, 0.05, 0.1]
    num_generations = 40
    weights = np.random.rand(len(df_train.columns))  # Random weights for each feature

    #run experiments
    results = []
    for pop_size in population_sizes:
        for crossover_type in crossover_types:
            for mutation_rate in mutation_rates:
                best_solution, best_fitness = genetic_algorithm(df_train, num_generations, pop_size, mutation_rate, crossover_type, weights)
                results.append({'Population Size': pop_size, 'Crossover Type': crossover_type, 'Mutation Rate': mutation_rate,
                                'Best Solution': best_solution, 'Best Fitness': best_fitness})

    #results to a CSV file
    results_df = pd.DataFrame(results)
    results_df.to_csv('exp1_results.csv', index=False)


**FROM TRAINING AND TESTING DATASETS**

After running experiments on the training dataset and saving the results, we evaluate the best solutions obtained from the training on the testing dataset.

In [1]:
import numpy as np
import pandas as pd

#chromosome representation
def create_chromosome(num_features):
    return np.random.choice([0, 1], size=num_features)


def fitness_function(chromosome, df):
    selected_features = df.columns[chromosome == 1]
    fitness = df[selected_features].sum().sum()
    return fitness

#selection function
def selection(population, fitness_values):
    probabilities = fitness_values / np.sum(fitness_values)
    selected_indices = np.random.choice(len(population), size=len(population), p=probabilities)
    selected_population = [population[i] for i in selected_indices]
    return selected_population

#crossover function
def one_point_crossover(parent1, parent2):
    crossover_point = np.random.randint(1, len(parent1))
    child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
    return child1, child2

def two_point_crossover(parent1, parent2):
    crossover_points = sorted(np.random.choice(len(parent1), size=2, replace=False))
    child1 = np.concatenate((parent1[:crossover_points[0]], parent2[crossover_points[0]:crossover_points[1]], parent1[crossover_points[1]:]))
    child2 = np.concatenate((parent2[:crossover_points[0]], parent1[crossover_points[0]:crossover_points[1]], parent2[crossover_points[1]:]))
    return child1, child2

#mutation function
def mutation(chromosome, mutation_rate):
    for i in range(len(chromosome)):
        if np.random.random() < mutation_rate:
            chromosome[i] = 1 - chromosome[i]
    return chromosome

#initialize population
def initialize_population(population_size, num_features):
    population = [create_chromosome(num_features) for _ in range(population_size)]
    return population

#algorithm
def genetic_algorithm(df, num_generations, population_size, mutation_rate, crossover_type):
    num_features = len(df.columns)
    population = initialize_population(population_size, num_features)

    crossover_fn = one_point_crossover if crossover_type == 'one-point' else two_point_crossover

    for generation in range(num_generations):
        fitness_values = [fitness_function(chromosome, df) for chromosome in population]
        selected_population = selection(population, fitness_values)

        offspring = []
        for i in range(0, len(selected_population), 2):
            parent1, parent2 = selected_population[i], selected_population[i+1]
            child1, child2 = crossover_fn(parent1, parent2)
            offspring.extend([child1, child2])

        mutated_offspring = [mutation(chromosome, mutation_rate) for chromosome in offspring]

        population = mutated_offspring

    final_fitness_values = [fitness_function(chromosome, df) for chromosome in population]
    best_index = np.argmax(final_fitness_values)
    best_solution = population[best_index]
    best_fitness = final_fitness_values[best_index]

    return best_solution, best_fitness

#preprocessing function to handle categorical columns
def preprocess_data(df):
    categorical_columns = df.select_dtypes(include=['category']).columns
    df = df.drop(categorical_columns, axis=1)
    return df

if __name__ == "__main__":
    #training dataset
    df_train = pd.read_parquet('UNSW_NB15_training-set.parquet')

    #testing dataset
    df_test = pd.read_parquet('UNSW_NB15_testing-set.parquet')

    #preprocess both datasets
    df_train = preprocess_data(df_train)
    df_test = preprocess_data(df_test)

    #define parameters for experiments
    population_sizes = [50, 100, 200]
    crossover_types = ['one-point', 'two-point']
    mutation_rates = [0.01, 0.05, 0.1]
    num_generations = 50

    #run experiments on training dataset
    train_results = []
    for pop_size in population_sizes:
        for crossover_type in crossover_types:
            for mutation_rate in mutation_rates:
                best_solution, best_fitness = genetic_algorithm(df_train, num_generations, pop_size, mutation_rate, crossover_type)
                train_results.append({'Population Size': pop_size, 'Crossover Type': crossover_type, 'Mutation Rate': mutation_rate,
                                'Best Solution': best_solution, 'Best Fitness': best_fitness})

    #save training results to a CSV file
    train_results_df = pd.DataFrame(train_results)
    train_results_df.to_csv('train_results.csv', index=False)

    #evaluate on testing dataset
    test_results = []
    for idx, row in train_results_df.iterrows():
        best_solution = row['Best Solution']
        test_fitness = fitness_function(best_solution, df_test)
        test_results.append({'Population Size': row['Population Size'], 'Crossover Type': row['Crossover Type'],
                             'Mutation Rate': row['Mutation Rate'], 'Test Fitness': test_fitness})

    #save testing results to a CSV file
    test_results_df = pd.DataFrame(test_results)
    test_results_df.to_csv('test_results.csv', index=False)
